In [0]:
# run if haversine not already installed
# %pip install haversine 

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, count, dayofweek, explode, format_number, hour, lit, lower, regexp_replace, row_number, size, split, sum, udf
from pyspark.ml.feature import NGram
from pyspark.sql.types import StringType, FloatType
from pyspark.sql.window import Window
from haversine import haversine as hs

spark = SparkSession.builder.appName("dh3382-hw2").getOrCreate()

BAKERY_PATH = '/FileStore/tables/BreadBasket_DMS.csv'
POP_PATH = '/FileStore/tables/populationbycountry19802010millions.csv'
REST_PATH = '/FileStore/tables/Restaurants_in_Durham_County_NC.csv'
REST_FORECLOSED_PATH = '/FileStore/tables/durham_nc_foreclosure_2006_2016.json'
WORD_DIR_PATH = '/FileStore/tables/hw1text/'


## Question 1: Show total number bought per item, per day, between 11am and 1pm

Program first reads in the BreadBasket_DMS csv file and filters out all NONE entries before moving on to part 1-specific processing. It proceeds to filter out all transactions outside of the specified time range (11:00-13:00 inclusive), and then groups the result first by Item, then by Date, and creates a "Quantity" column by counting the entries. This results in a dataframe that show the total number of each item bought per day between 11:00 and 13:00 inclusive

In [0]:
# read and clean bakery data
bakery_data = spark.read.options(header=True, inferSchema=True).csv(BAKERY_PATH)

# remove NONE entries
bakery_data = bakery_data.filter(col('Item') != 'NONE')

In [0]:
##### QUESTION 1: Show the total number bought by item, per day, between 11AM and 1PM #####

# filter out all transactions outside of specified time range (11:00-13:00 inclusive)
bakery_data_lunch = bakery_data.filter(col('Time').between('11:00:00', '13:00:00'))

# group by Item and Day purchased, then get total count of items per day purchased between 11:00-13:00
bakery_q1_res = bakery_data_lunch.groupBy(col('Item'), col('Date')).count().withColumnRenamed('count', 'Quantity')

#sort by item, date for readability
bakery_q1_res = bakery_q1_res.sort('Item', 'Date')

# DF with answer to question 1
bakery_q1_res.show()

+--------------------+----------+--------+
|                Item|      Date|Quantity|
+--------------------+----------+--------+
|Afternoon with th...|2017-01-21|       2|
|Afternoon with th...|2017-01-22|       1|
|Afternoon with th...|2017-02-18|       1|
|           Alfajores|2016-11-02|       1|
|           Alfajores|2016-11-04|       1|
|           Alfajores|2016-11-08|       3|
|           Alfajores|2016-11-11|       3|
|           Alfajores|2016-11-12|       3|
|           Alfajores|2016-11-13|       1|
|           Alfajores|2016-11-17|       5|
|           Alfajores|2016-11-20|       4|
|           Alfajores|2016-11-25|       2|
|           Alfajores|2016-11-27|       2|
|           Alfajores|2016-12-03|       1|
|           Alfajores|2016-12-04|       2|
|           Alfajores|2016-12-05|       1|
|           Alfajores|2016-12-07|       1|
|           Alfajores|2016-12-09|       1|
|           Alfajores|2016-12-13|       1|
|           Alfajores|2016-12-14|       2|
+----------


## Question 2: Show the top 3 (by qty) items bought by Daypart, by DayType

The first cell (Cmd 7) defines the UDFs that will be used to create the columns Daypart (Weekend/Weekday) and Daytype (Morning/Afternoon/Night). It then uses these UDFs to add the columns to the bakery_data DF that already had the NONE entries removed. The next step groups this DF by Daypart, Daytime, and Item to count the number of each item purchased for every possible Daytime/Daypart combination. A window function is then used to partition by Daypart/Daytime and order by total purchases per item, with the output limited to the top 3 purchases for each Daypart/Daytime. The last two transformations modify the format so that the top 3 items per Daypart/Daytime are collected in a list and displayed on a single row, and the final transformation reorders the columns for readability before calling show()


In [0]:
##### QUESTION 2: Show the top 3 (by qty) items bought by Daypart, by DayType #####

#udf to define weekend/weekday
def weekend(is_weekend):
    if is_weekend:
        return "Weekend"
    else:
        return "Weekday"

weekend_udf = udf(lambda is_weekend: weekend(is_weekend), StringType() )

# udf to define part of day
def day_part(date):
    if (date.hour >= 7) & (date.hour < 12):
        return "Morning"
    elif (date.hour >= 12) & (date.hour < 17):
        return "Afternoon"
    else:
        return "Night"

day_part_udf = udf(lambda date: day_part(date), StringType() )

In [0]:
# use weekend udf to add Daytype column, pass in boolean with sparkSQL dayofweek and isin functions
bakery_weekend = bakery_data.withColumn('Daytype', weekend_udf(dayofweek(col('Date') ).isin([1, 7]) ) )

# use day_part udf to add Daypart column
bakery_daypart = bakery_weekend.withColumn('Daypart', day_part_udf(col('Time') ) )

#group by Daypart and Daytype and count purchases
bakery_purchases = bakery_daypart.groupBy(col('Daypart'), col("Daytype"), col('Item') )\
  .count()\
  .withColumnRenamed('count', 'Purchases')

# partition by Daypart, Daytype and find top 3 Purchases for each Daypart/Daytime combination
windowBakery = Window.partitionBy('Daypart', 'Daytype')\
  .orderBy(col('Purchases').desc() )

bakery_top_purchases = bakery_purchases.withColumn("row",row_number()\
  .over(windowBakery) ) \
  .filter(col("row") <= 3) \

# Collect all items into single row grouped by Daypart/Daytype
bakery_q2_res = bakery_top_purchases.groupBy(col('Daypart'), col("Daytype") )\
  .agg(collect_list('Item').alias('Top_3_Items') )

# Select columns in order Daypart/Top_3_Items/Daytype for cleaner output format
bakery_q2_res.select(col('Daypart'), col('Top_3_Items'), col('Daytype') ).show(truncate=False)


+---------+------------------------------------------+-------+
|Daypart  |Top_3_Items                               |Daytype|
+---------+------------------------------------------+-------+
|Afternoon|[Coffee, Bread, Tea]                      |Weekday|
|Afternoon|[Coffee, Bread, Tea]                      |Weekend|
|Morning  |[Coffee, Bread, Pastry]                   |Weekday|
|Morning  |[Coffee, Bread, Pastry]                   |Weekend|
|Night    |[Coffee, Bread, Tea]                      |Weekday|
|Night    |[Coffee, Tshirt, Afternoon with the baker]|Weekend|
+---------+------------------------------------------+-------+




## Question 3: The total number of entities by “rpt_area_desc”

The first cell (Cmd 10) reads in the European style CSV file, defining the semicolon delimiter in read options. After dropping any entries with a null Rpt_Area_desc entry, the resulting DF is then simply grouped by the Rpt_Area_Desc column, counted to a column 'Total', and sorted in descending order. The final DF is then shown with the top 20 results

In [0]:
##### QUESTION 3: The total number of entities by “rpt_area_desc” #####
# set European style csv delimiter (;)
rest_data_raw = spark.read.options(header=True, delimiter=';', inferSchema=True)\
    .csv(REST_PATH)

In [0]:
# group by rpt area desc, remove null entries, and aggregate count as 'Total'
rest_q3_res = rest_data_raw\
    .na.drop(subset=['Rpt_Area_Desc'])\
    .groupBy(col('Rpt_Area_Desc') )\
    .agg(count('*').alias('Total') )\

rest_q3_res.show()

+--------------------+-----+
|       Rpt_Area_Desc|Total|
+--------------------+-----+
|  Bed&Breakfast Home|    4|
|        Summer Camps|    4|
|        Institutions|   30|
|   Local Confinement|    2|
|         Mobile Food|  147|
|    School Buildings|   89|
|         Summer Food|  242|
|      Swimming Pools|  420|
|            Day Care|  173|
|Tattoo Establishm...|   32|
|    Residential Care|  154|
|   Bed&Breakfast Inn|    2|
|      Adult Day Care|    5|
|             Lodging|   62|
|        Food Service| 1093|
+--------------------+-----+




## Question 4: Show the top 10 regions with the biggest percentage decrease in population, for the years 1990-2000

The first cell (Cmd 13) is dedicated to data cleanup. The column name for region was missing from the header, so the columns attribute is used to find the default name and rename the first column to 'Region'. Some entries included either 'NA' or '--' population data, so these entries were filtered out to leave only entries with valid population numbers. Filling in a random value when one was not available did not seem reasonable, and '--' seems to apply to countries and regions that no longer exist. If a country did not exist for the entire range 1990-2000, it does not seem reasonable to assert it had a population decrease during that specific range. Furthermore, there were 7 aggregate regions listed. The Former USSR was already removed by the previous filter, but the other 6 had to be manually filtered out to leave only country-level data as specified during class.

After cleaning, I first calculated the gross increase for each region and created a new column 'gross_increase'. I then filtered out all positive population increases as they would not be among the top 10 population decreases, and this could potentially save significant compute time on a larger dataset. I then calculated the percent increases, saved them in a column 'perc_increase_1990_2000', and sorted the data on 'perc_increase_1990_2000' while saving only the Region and perc_increase_1990_2000 columns to a new dataframe. Note that a negative perc_increase denotes a population decrease. For the final result, I showed the top 10 results from this sorted DF.

In [0]:
#####  QUESTION 4: Show the top 10 regions with the biggest percentage decrease in population, for the years 1990-2000 #####

pop_data_raw = spark.read.options(header=True, inferSchema=True)\
    .csv(POP_PATH)

# first column name is Null in file, find col name through columns attribute and rename first column to region
cols = pop_data_raw.columns 
pop_data_clean_header = pop_data_raw.select(col(cols[0]).alias('Region'), col('1990'), col('2000') )

# remove regions with no population data (e.g. Antarctica, Wake Island, Croatia, Former U.S.S.R.)
pop_data_clean_pop_nums = pop_data_clean_header.filter((col('1990')  != 'NA') &  (col('2000')  != 'NA') )
pop_data_clean_pop_nums = pop_data_clean_pop_nums.filter((col('1990')  != '--') &  (col('2000')  != '--') )

# remove leftover aggregate regions
pop_data_clean_regions = pop_data_clean_pop_nums\
    .filter((col('Region') != 'World') & (col('Region') != 'North America') & (col('Region') != 'Central & South America') & (col('Region') != 'Eurasia') & (col('Region') != 'Western Sahara') & (col('Region') != 'Asia & Oceania') )

In [0]:
# calculate gross increase
pop_decrease_gross = pop_data_clean_regions.withColumn('gross_increase', col('2000') - col('1990') )

# remove regions with positive population growth
pop_decrease_gross = pop_decrease_gross.filter(col('gross_increase') <= 0)

# calculate percentage of growth increase, multiply float by 100 for readability
pop_decrease_perc = pop_decrease_gross.withColumn('perc_increase_1990_2000', (col('gross_increase')/col('1990') ) * 100 )

# create DF sorted in ascending order of pop increase, only contains Region and Percent Increase, formatted to 2 decimal
# points for readability
pop_q4_res = pop_decrease_perc.sort(col('perc_increase_1990_2000') )\
    .select(col('Region'), format_number(col('perc_increase_1990_2000'), 2 ).alias('Percent Increase (1990-2000)') )

# show top 10 results
pop_q4_res.show(10)

+-------------+----------------------------+
|       Region|Percent Increase (1990-2000)|
+-------------+----------------------------+
|   Montserrat|                      -63.19|
|     Bulgaria|                      -12.09|
| Cook Islands|                      -11.31|
| Sierra Leone|                       -9.91|
|       Kuwait|                       -7.84|
|    Gibraltar|                       -6.32|
|Faroe Islands|                       -3.48|
|      Albania|                       -2.67|
|      Hungary|                       -2.16|
|      Romania|                       -1.83|
+-------------+----------------------------+
only showing top 10 rows



## Question 5: Word Count

In the first cell (Cmd 16), I first read in the data and normalize the words to lower case, replacing all punctuation with white spaces. I then split the words values by spaces to create an array of single words and explode the column to create a DF where every single word from the input has its own row. I overwrite the original words column to avoid duplicate data when exploding. I then simply group by the words column and call count to create a new DF with all unique words and their respective number of occurrences ('count') and show the results.

In [0]:
##### QUESTION 5 #####
"""
Do word count exercise using pyspark. Ignore punctuation and normalize to lower case. Replace characters NOT in this set: [0-9a-z] with space.
"""
word_df = spark.read.text(WORD_DIR_PATH)

# normalize to lower case
word_df_lower = word_df.select(lower(col('value')).alias('words') )

# replace punctuation with space
word_df_parsed = word_df_lower.withColumn('words', regexp_replace('words', '[^a-z0-9]', ' ') )

In [0]:
# explode and overwrite words column to create row for each word
word_df_explode = word_df_parsed.withColumn('words', explode(split('words', " +") ) )

# group by words and count each occurence of each word
word_q5_res = word_df_explode.groupBy(col('words') ).count()

# show result
word_q5_res.show()

+------------+-----+
|       words|count|
+------------+-----+
|       trail|   57|
|       those| 3409|
|    medicare|   32|
|        some| 4335|
|         few| 1057|
|   connected|  162|
| herzegovina|    7|
|   involving|   99|
|    randomly|   10|
|     clinics|   78|
|       still| 2139|
| transmitted|   92|
|      travel| 1367|
|vicissitudes|    1|
|      online| 1357|
|         wto|   17|
|  paramedics|   26|
|          07|   90|
|   traveling|   97|
|   recognize|   66|
+------------+-----+
only showing top 20 rows



## Question 6: 10 Most Common Bigrams

In the first cell (Cmd 19), I transform the parsed words column, which was normalized and had its punctuation removed but had not yet been exploded, and convert each row into an array of strings using 'split'. This is a requirement for using the NGram feature from the pyspark.ml.feature library. I define the bigram transformation, as well as the input and output columns, before applying the transformation on this DF. The result is a DF where all rows are an array of bigrams as well as the original array of word strings. I remove the 'words' column to avoid unnecessary duplicate data before exploding the bigram column to obtain a DF where each row is a bigram. I then group by bigram and count in order to count the occurence of each unique bigrams in the DF. After this, I sort the resulting DF by the count column in descending order and show the first 10 columns to provide the solution

In [0]:
##### QUESTION 6: Find the 10 most common bigrams #####

# convert strings to arrays of strings for NGram transformation
word_df_str_arrays = word_df_parsed.withColumn('words', split('words', ' +') )

# transform word_df to bigram_df using NGram feature
bigram = NGram(n=2).setInputCol('words').setOutputCol('bigrams')
bigram_df = bigram.transform(word_df_str_arrays)

# remove unnecessary words column
bigram_df = bigram_df.select(col('bigrams') )

# explode bigrams so each has own separate row
bigram_df = bigram_df.withColumn('bigrams', explode(col('bigrams') ) )

# group by bigram and count occurences
bigram_df = bigram_df.groupBy(col('bigrams') ).count()

# sort on bigram count for q5 res
bigram_q5_res = bigram_df.sort('count', ascending=False)

# show top 10 bigrams
bigram_q5_res.show(10)

+--------+-----+
| bigrams|count|
+--------+-----+
|  of the|17484|
|  in the|12808|
|   p the|10363|
|covid 19| 8762|
|  to the| 8372|
| for the| 5588|
|     n t| 5393|
|  on the| 5032|
|   to be| 4581|
| will be| 4177|
+--------+-----+
only showing top 10 rows



## Question 7: Geolocations

I first defined a UDF to use in both parts a and b. It takes in two columns of coordinate strings, converts the strings to float tuples and passes them to the haversine function with the units defined as miles. It returns a column of the haversine distance between the two coordinates in miles

In [0]:
##### Question 7 #####
"""
a) Find the food service and active restaurant closest to the following coordinates: 35.994914, -78.897133
b) With that restaurant as your center point, find the number of foreclosures within a 1 mile radius
"""
# user function that accepts 2 strings of coordinates and outputs float distance in miles for part a
def hs_with_str(coord_str1, coord_str2):
    # convert coordinate strings to float tuples
    coord_tup1 = tuple(map(float, coord_str1.split(', ') ) )
    coord_tup2 = tuple(map(float, coord_str2.split(', ') ) )

    # plug float tuples intro haversine function, specify unit as miles
    return hs(coord_tup1, coord_tup2, unit='mi')

# convert hs_with_str to udf
hs_with_str_udf = udf(lambda coord1, coord2: hs_with_str(coord1, coord2), FloatType() )

### Part A: Find the food service and active restaurant closest to the following coordinates: 35.994914, -78.897133

I first filter the rest_data_raw DF from Q3 to define a new DF with only the active, food service restaurants. I then drop any rows with null geolocation values, as these null values would break the haversine UDF and there is no reasonable replacement data in this case. I then use the haversine UDF to create a new column, 'Distance(mi)', passing in a column of the origin coordinates using the lit() function as well as the geolocation column. I obtain the final results by sorting this DF, and then show the top result with only the columns 'Premise_name', 'Status', 'Rpt_Area_Desc', 'geolocation', and 'Distance(mi)' for readability.

In [0]:
## PART A: Find the food service and active restaurant closest to the following coordinates: 35.994914, -78.897133

# origin coordinate string specified in prompt
ORIGIN_COORD_STR = '35.994914, -78.897133'

#filter dataset to include only active food service restaurants
rest_data_active_fs = rest_data_raw.filter(col('Status') == 'ACTIVE').filter(col('Rpt_Area_Desc') == 'Food Service')

# drop any restaurants with no geolocation data
rest_data_active_fs = rest_data_active_fs.na.drop(subset=['geolocation'])

# apply haversine UDF to create Distance(mi) column
rest_data_distance = rest_data_active_fs\
    .withColumn('Distance(mi)', hs_with_str_udf(col('geolocation'), lit(ORIGIN_COORD_STR) ) )

# sort on Distance(mi) for final result
rest_q7a_res = rest_data_distance.sort('Distance(mi)')

# show closest restaurant, select only relevant columns for cleaner output
rest_q7a_res\
    .select(col('Premise_Name'), col('Status'), col('Rpt_Area_Desc'), col('geolocation'), col('Distance(mi)') )\
    .show(1, truncate=False)

+------------------------+------+-------------+-----------------------+------------+
|Premise_Name            |Status|Rpt_Area_Desc|geolocation            |Distance(mi)|
+------------------------+------+-------------+-----------------------+------------+
|OLD HAVANA SANDWICH SHOP|ACTIVE|Food Service |35.9932826, -78.8981331|0.1258222   |
+------------------------+------+-------------+-----------------------+------------+
only showing top 1 row



### Part B: With that restaurant as your center point, find the number of foreclosures within a 1 mile radius

I save the coordinates from the part a's result using collect(), and then read in the json dataset. I create a DF with only the geocode subfield from this dataset, as that is all that is required to answer the question. I again remove rows with null values for coordinates. Because my original haversine UDF accepts strings, I cast the geocode column, originally a list of floats, to a string column and remove the brackets. The list values would require some casting either way to be able to use the haversine function, so this seemed to be an acceptable, though somewhat bulky, solution. I again create a 'Distance(mi)' column in the same way as part A, and proceed to filter out all distances over 1 mile. For the final result, I simply print out the count of the resulting DF

In [0]:
## PART B: With that restaurant as your center point, find the number of foreclosures within a 1 mile radius

# save coordinates of restaurant from Q7a to variable COORD_STR
CENTER_COORD_STR = rest_q7a_res.collect()[0]['geolocation']

# read in foreclosure dataset
fc_rest_raw = spark.read.json(REST_FORECLOSED_PATH)

# select only geocode subfield and discard unnecessary data
fc_rest_coords = fc_rest_raw.select(col('fields').getItem('geocode').alias('geocodes') )

# remove entries with null geolocation values
fc_rest_coords = fc_rest_coords.na.drop()

# cast geocode column to string
fc_coords_str = fc_rest_coords.withColumn('geocodes', col('geocodes').cast(StringType() ) )

# remove brackets
fc_coords_str = fc_coords_str.withColumn('geocodes', regexp_replace('geocodes', "[\[\]]", "") )

# calculate Distance(mi) column using hs_with_str_udf
fc_coords_distance = fc_coords_str.withColumn('Distance(mi)', hs_with_str_udf(col('geocodes'), lit(CENTER_COORD_STR) ) )

#filter out all restaurants more than 1 mile from center point
fc_coords_q7b_res = fc_coords_distance.filter(col('Distance(mi)') <= 1)

# count and print umber of foreclosures within 1 mile of center point
print("RESULT")
print('Number of foreclosures within 1 mile of center point: %d' % fc_coords_q7b_res.count() )

RESULT
Number of foreclosures within 1 mile of center point: 320
